# Comparison with the R version of `tximport`

This file is included to provide the code used to generate the output from `tximport` that the test in `test_correctness.py` compares against. It will note be automatically run by `pytest` and the assertions provided at the end are redundant, since they are already included in `test_correctness.py`.

In [1]:
!pip install rpy2==3.4.5 -q

In [2]:
!python3 -m rpy2.situation

rpy2 version:
3.4.5
Python version:
3.9.16 (main, Sep 11 2024, 11:23:52) 
[Clang 15.0.0 (clang-1500.3.9.4)]
Looking for R's HOME:
    Environment variable R_HOME: None
    Calling `R RHOME`: /Library/Frameworks/R.framework/Resources
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:

R version:
    In the PATH: R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/Library/Frameworks/R.framework/Resources/include']
  libraries:
  ['pcre2-8', 'lzma', 'bz2', 'z', 'icucore', 'dl', 'm', 'iconv']
  library_dirs:
  ['/opt/R/arm64/lib', '/opt/R/arm64/lib']
  extra_compile_args:
  []
  extra_link_args:
  ['-F/Library/Frameworks/R.framework/..', '-framework', 'R']


In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
R.version.string

[1] "R version 4.3.1 (2023-06-16)"


In [5]:
%%R
install.packages("matrixStats")

--- Bitte einen CRAN-Spiegelserver für diese Sitzung auswählen ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Austria [https]
 6: Belgium (Brussels) [https]
 7: Brazil (PR) [https]
 8: Brazil (SP 1) [https]
 9: Brazil (SP 2) [https]
10: Bulgaria [https]
11: Canada (MB) [https]
12: Canada (ON 1) [https]
13: Canada (ON 2) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Guangzhou) [https]
20: China (Jinan) [https]
21: China (Nanjing) [https]
22: China (Shanghai 2) [https]
23: China (Shenzhen) [https]
24: China (Wuhan) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: France (Lyon 1) [https]
33: France (Lyon 2) [https]
34: France (Marseille) 

R[write to console]: versuche URL 'https://ftp.gwdg.de/pub/misc/cran/bin/macosx/big-sur-arm64/contrib/4.3/matrixStats_1.4.1.tgz'

R[write to console]: Content type 'application/octet-stream'
R[write to console]:  length 618600 bytes (604 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 


Die heruntergeladenen Binärpakete sind in 
	/var/folders/vg/vt1jjh256dn1g4sd6pknwxbs_qmnws/T//RtmpMZUxWz/downloaded_packages


In [6]:
%%R
library(tximport)
library(readr)
library(matrixStats)
dir <- "./data/fabry_disease"
tx2gene <- read_tsv(file.path(dir, "transcript_gene_mapping_human.tsv"))
files <- c(
    file.path(dir, "SRR16504309_wt/quant.sf"),
    file.path(dir, "SRR16504310_wt/quant.sf"),
    file.path(dir, "SRR16504311_ko/quant.sf"),
    file.path(dir, "SRR16504312_ko/quant.sf")
)
countsFromAbundanceOptions <- c("no", "scaledTPM", "lengthScaledTPM")
for (idx in seq_along(countsFromAbundanceOptions)) {
    txi <- tximport(
        files,
        type = "salmon",
        tx2gene = tx2gene,
        countsFromAbundance = countsFromAbundanceOptions[idx],
        dropInfReps = TRUE,
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport.csv")
    if (!is.null(countsFromAbundanceOptions[idx])) {
        writePath <- gsub(".csv", paste0("_", countsFromAbundanceOptions[idx], ".csv"), writePath)
    }
    write.csv(txi$counts, writePath)
}
dataTypeOptions <- c("kallisto", "salmon")
for (idx in seq_along(dataTypeOptions)) {

    if (dataTypeOptions[idx] == "kallisto") {
        files <- c(
            file.path(dir, "SRR16504309_wt/abundance.h5"),
            file.path(dir, "SRR16504310_wt/abundance.h5"),
            file.path(dir, "SRR16504311_ko/abundance.h5"),
            file.path(dir, "SRR16504312_ko/abundance.h5")
        )
    } else {
        files <- c(
            file.path(dir, "SRR16504309_wt/quant.sf"),
            file.path(dir, "SRR16504310_wt/quant.sf"),
            file.path(dir, "SRR16504311_ko/quant.sf"),
            file.path(dir, "SRR16504312_ko/quant.sf")
        )
    }

    txi <- tximport(
        files,
        type = dataTypeOptions[idx],
        tx2gene = tx2gene,
        countsFromAbundance = "no",
        dropInfReps = FALSE,
        infRepStat = rowMedians,
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport_bootstrap.csv")
    writePath <- gsub(".csv", paste0("_", dataTypeOptions[idx], ".csv"), writePath)
    write.csv(txi$counts, writePath)
}
for (idx in seq_along(dataTypeOptions)) {

    if (dataTypeOptions[idx] == "kallisto") {
        files <- c(
            file.path(dir, "SRR16504309_wt/abundance.h5"),
            file.path(dir, "SRR16504310_wt/abundance.h5"),
            file.path(dir, "SRR16504311_ko/abundance.h5"),
            file.path(dir, "SRR16504312_ko/abundance.h5")
        )
    } else {
        files <- c(
            file.path(dir, "SRR16504309_wt/quant.sf"),
            file.path(dir, "SRR16504310_wt/quant.sf"),
            file.path(dir, "SRR16504311_ko/quant.sf"),
            file.path(dir, "SRR16504312_ko/quant.sf")
        )
    }

    txi <- tximport(
        files,
        type = dataTypeOptions[idx],
        tx2gene = tx2gene,
        countsFromAbundance = "no",
        txOut = TRUE,
        dropInfReps = FALSE,
        infRepStat = rowMedians,
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport_bootstrap_transcripts.csv")
    writePath <- gsub(".csv", paste0("_", dataTypeOptions[idx], ".csv"), writePath)
    write.csv(txi$counts, writePath)
}

Rows: 244191 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): transcript_id, gene_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 31380

R[write to console]: summarizing abundance

R[write to console]: summarizing counts

R[write to console]: summarizing length

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 31380

R[write to console]: summarizing abundance

R[write to console]: summarizing counts

R[write to console]: summarizing length

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 31380

R[write to console]: summarizing abundance

R[

In [7]:
!pytximport -i ./data/fabry_disease/SRR16504309_wt/quant.sf -i ./data/fabry_disease/SRR16504310_wt/quant.sf -i ./data/fabry_disease/SRR16504311_ko/quant.sf -i ./data/fabry_disease/SRR16504312_ko/quant.sf -t salmon -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -ow -o ./data/fabry_disease/counts_pytximport_no.csv
!pytximport -i ./data/fabry_disease/SRR16504309_wt/quant.sf -i ./data/fabry_disease/SRR16504310_wt/quant.sf -i ./data/fabry_disease/SRR16504311_ko/quant.sf -i ./data/fabry_disease/SRR16504312_ko/quant.sf -t salmon -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -ow -o ./data/fabry_disease/counts_pytximport_scaledTPM.csv -c scaled_tpm
!pytximport -i ./data/fabry_disease/SRR16504309_wt/quant.sf -i ./data/fabry_disease/SRR16504310_wt/quant.sf -i ./data/fabry_disease/SRR16504311_ko/quant.sf -i ./data/fabry_disease/SRR16504312_ko/quant.sf -t salmon -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -ow -o ./data/fabry_disease/counts_pytximport_lengthScaledTPM.csv -c length_scaled_tpm

2024-09-11 15:57:10,820: Starting the import.
Reading quantification files: 4it [00:01,  3.01it/s]
2024-09-11 15:57:12,265: Converting transcript-level expression to gene-level expression.
2024-09-11 15:57:12,701: Not all transcripts are present in the mapping. 31380 out of 253181 missing. Removing the missing transcripts.
2024-09-11 15:57:12,991: Matching gene_ids.
2024-09-11 15:57:13,168: Creating gene abundance.
2024-09-11 15:57:13,470: Creating gene counts.
2024-09-11 15:57:13,471: Creating lengths.
2024-09-11 15:57:13,595: Replacing missing lengths.
2024-09-11 15:57:13,680: Creating gene expression dataset.
2024-09-11 15:57:13,710: Saving the gene-level expression to: data/fabry_disease/counts_pytximport_no.csv.
2024-09-11 15:57:13,784: Finished the import in 2.96 seconds.
2024-09-11 15:57:14,837: Starting the import.
Reading quantification files: 4it [00:01,  2.99it/s]
2024-09-11 15:57:16,319: Converting transcript-level expression to gene-level expression.
2024-09-11 15:57:16,74

In [8]:
import pandas as pd

counts_tximport_no = pd.read_csv("./data/fabry_disease/counts_tximport_no.csv")
counts_tximport_scaledTPM = pd.read_csv("./data/fabry_disease/counts_tximport_scaledTPM.csv")
counts_tximport_lengthScaledTPM = pd.read_csv("./data/fabry_disease/counts_tximport_lengthScaledTPM.csv")

counts_pytximport_no = pd.read_csv("./data/fabry_disease/counts_pytximport_no.csv")
counts_pytximport_scaledTPM = pd.read_csv("./data/fabry_disease/counts_pytximport_scaledTPM.csv")
counts_pytximport_lengthScaledTPM = pd.read_csv("./data/fabry_disease/counts_pytximport_lengthScaledTPM.csv")
counts_pytximport_no.columns = counts_tximport_no.columns
counts_pytximport_scaledTPM.columns = counts_tximport_scaledTPM.columns
counts_pytximport_lengthScaledTPM.columns = counts_tximport_lengthScaledTPM.columns

pd.testing.assert_frame_equal(counts_tximport_no, counts_pytximport_no)
pd.testing.assert_frame_equal(counts_tximport_scaledTPM, counts_pytximport_scaledTPM)
pd.testing.assert_frame_equal(counts_tximport_lengthScaledTPM, counts_pytximport_lengthScaledTPM)

## Compare outputs for transcript-level summarization

In [9]:
%%R
dir <- "./data/salmon"
files_protein_coding <- c(
  file.path(dir, "quant.sf")
)
tx2gene <- read_tsv(file.path("./data/fabry_disease", "transcript_gene_mapping_human.tsv"))
countsFromAbundanceOptions <- c("scaledTPM", "dtuScaledTPM")
for (idx in seq_along(countsFromAbundanceOptions)) {
    txi <- tximport(
        files_protein_coding,
        type = "salmon",
        tx2gene = tx2gene,
        txOut = TRUE,
        countsFromAbundance = countsFromAbundanceOptions[idx],
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport.csv")
    if (!is.null(countsFromAbundanceOptions[idx])) {
        writePath <- gsub(".csv", paste0("_", countsFromAbundanceOptions[idx], ".csv"), writePath)
    }
    write.csv(txi$counts, writePath)
}

Rows: 244191 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): transcript_id, gene_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 



In [10]:
!pytximport -i ./data/salmon/quant.sf -m ./data/fabry_disease/transcript_gene_mapping_human.tsv  -ow  -o ./data/salmon/counts_pytximport_dtuScaledTPM.csv -t salmon -tx -c dtu_scaled_tpm

2024-09-11 15:57:23,917: Starting the import.
Reading quantification files: 1it [00:00, 295.69it/s]
2024-09-11 15:57:24,266: Recreating transcript counts from abundances.
2024-09-11 15:57:24,266: Setting counts to length scaled TPM.
2024-09-11 15:57:24,267: Saving the gene-level expression to: data/salmon/counts_pytximport_dtuScaledTPM.csv.
2024-09-11 15:57:24,270: Finished the import in 0.35 seconds.


In [11]:
counts_tximport_dtuScaledTPM = pd.read_csv("./data/salmon/counts_tximport_dtuScaledTPM.csv", index_col=0).sort_index()
counts_pytximport_dtuScaledTPM = pd.read_csv(
    "./data/salmon/counts_pytximport_dtuScaledTPM.csv", index_col=0
).sort_index()
# cut the transcript version from the index
counts_tximport_dtuScaledTPM.index = counts_tximport_dtuScaledTPM.index.str.split(".").str[0]
counts_pytximport_dtuScaledTPM.columns = counts_tximport_dtuScaledTPM.columns

pd.testing.assert_frame_equal(counts_tximport_dtuScaledTPM, counts_pytximport_dtuScaledTPM)

## Compare outputs for RSEM

In [12]:
%%R
dir <- "./data/rsem"
files_protein_coding <- c(
  file.path(dir, "test.genes.results.gz")
)
tx2gene <- read_tsv(file.path("./data/fabry_disease", "transcript_gene_mapping_human.tsv"))
countsFromAbundanceOptions <- c("no")
for (idx in seq_along(countsFromAbundanceOptions)) {
    txi <- tximport(
        files_protein_coding,
        type = "rsem",
        tx2gene = tx2gene,
        txIn = FALSE,
        countsFromAbundance = countsFromAbundanceOptions[idx],
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport.csv")
    if (!is.null(countsFromAbundanceOptions[idx])) {
        writePath <- gsub(".csv", paste0("_", countsFromAbundanceOptions[idx], ".csv"), writePath)
    }
    write.csv(txi$counts, writePath)
}

Rows: 244191 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): transcript_id, gene_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 

